In [7]:
import pandas as pd
import numpy as np
from typing import Dict, Any
import random

In [2]:
prices = pd.read_csv("../app/data/sample_prices.csv", parse_dates=["date"])
prices.head()


,date,ticker,close,volume
0,2025-01-02,AAPL,172.30,52000000
1,2025-01-03,AAPL,173.10,48200000
2,2025-01-06,AAPL,171.85,49500000
3,2025-01-07,AAPL,174.25,51000000
4,2025-01-08,AAPL,175.00,53000000


In [3]:
class MarketAgent:
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def run(self, ticker: str) -> Dict[str, Any]:
        df = self.data[self.data["ticker"] == ticker].sort_values("date")
        if df.empty:
            return {"error": f"Ticker {ticker} not found"}
        
        df["ret"] = df["close"].pct_change()
        features = {
            "last_price": df["close"].iloc[-1],
            "returns_7": df["ret"].tail(7).mean(),
            "vol_7": df["ret"].tail(7).std(),
            "sma_5": df["close"].tail(5).mean(),
            "sma_20": df["close"].tail(20).mean(),
        }
        return {"ticker": ticker, "features": features}


In [11]:
class AnalystAgent:
    def run(self, features: Dict[str, Any]) -> Dict[str, Any]:
        r, vol = features["returns_7"], features["vol_7"]
        sma5, sma20 = features["sma_5"], features["sma_20"]

        score = 0
        if r > 0.002: score += 1
        if sma5 > sma20: score += 1
        if vol < 0.02: score += 1

        if score >= 2:
            action = "BUY"
        elif score == 1:
            action = "HOLD"
        else:
            action = "SELL"

        explanation = f"Score={score} | Ret7={r:.4f}, Vol7={vol:.4f}, SMA5={sma5:.2f}, SMA20={sma20:.2f}"
        return {"action": action, "explanation": explanation, "confidence": 0.6 + 0.1*score}


In [9]:
class CriticAgent:
    def run(self, analysis: Dict[str, Any]) -> Dict[str, Any]:
        action = analysis["action"]
        confidence = analysis["confidence"]

        critique = []
        if confidence < 0.7:
            critique.append("Confiança baixa, resultado incerto.")
        if action == "BUY" and confidence < 0.8:
            critique.append("A recomendação de compra não é forte.")
        if action == "SELL" and "SMA5" in analysis.get("explanation", ""):
            critique.append("Verifique tendência de curto prazo antes de vender.")

        return {"valid": len(critique) == 0, "critique": critique}

In [6]:
class FinancialAdvisorSystem:
    def __init__(self, market_agent, analyst_agent, critic_agent):
        self.market_agent = market_agent
        self.analyst_agent = analyst_agent
        self.critic_agent = critic_agent

    def run(self, ticker: str) -> Dict[str, Any]:
        # 1. Market Agent
        market_result = self.market_agent.run(ticker)
        if "error" in market_result:
            return market_result
        
        # 2. Analyst Agent
        analysis = self.analyst_agent.run(market_result["features"])
        
        # 3. Critic Agent
        review = self.critic_agent.run(analysis)
        
        return {
            "ticker": ticker,
            "features": market_result["features"],
            "analysis": analysis,
            "review": review
        }


In [12]:
market_agent = MarketAgent(prices)
analyst_agent = AnalystAgent()
critic_agent = CriticAgent()

advisor = FinancialAdvisorSystem(market_agent, analyst_agent, critic_agent)

result = advisor.run("AAPL")
print(result)


{'ticker': 'AAPL', 'features': {'last_price': 188.5, 'returns_7': 0.003076245327217048, 'vol_7': 0.004177539702512538, 'sma_5': 187.21999999999997, 'sma_20': 183.19}, 'analysis': {'action': 'BUY', 'explanation': 'Score=3 | Ret7=0.0031, Vol7=0.0042, SMA5=187.22, SMA20=183.19', 'confidence': 0.9}, 'review': {'valid': True, 'critique': []}}
